In [25]:
# Creating a model
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# Custom activation function
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects



#r = tf.cond(tf.less(x, y), f1, f2)


@tf.function
def custom_activation(x):
  #def f1(): return 0.001
  def f3(): return K.tf.add(0.5,K.tf.multiply(0.151,x))
  def f4(): return K.tf.add(0.908,K.tf.multiply(0.015,x))
  def f2(): return K.tf.where(K.tf.logical_and(K.tf.greater(x, -5), K.tf.less(x, 5)),x=f3(),y=f4())
  


  r = K.tf.where(
         condition=(K.tf.logical_and(K.tf.greater(x, -10), K.tf.less(x, -5))),
         x=K.tf.add(0.008,K.tf.multiply(0.001,x)),
         y=f2()
     )


  return r
"""
  r = K.tf.where(
         condition=(K.tf.logical_and(K.tf.greater(x, -10), K.tf.less(x, -5))),
         x=K.tf.add(0.001,K.tf.add(x,7.5)),
         y=f2()
     )
"""
#tf.logical_or((tf.logical_and(tf.greater(x, -10), tf.less(x, -5))), (tf.logical_and(tf.greater(x, -5), tf.less(x, 5)) )


    #def f1(): return tf.add(0.5,tf.multi(0.099,x))

    # return tf.where(
    #     condition=tf.logical_or((tf.logical_and(tf.greater(x, -10), tf.less(x, -5))), (tf.logical_and(tf.greater(x, -5), tf.less(x, 5)) )),
    #     x=tf.add(0.001,tf.add(x,7.5)),
    #     y=tf.add(0.5,tf.multiply(0.099,x))
    # )

"""
  x =tf.dtypes.cast(x, tf.int32)
  if (x > -10 & x < -5):
    y = 0.001 + 0.015*(x+7.5);
  if (x > -5 & x < 5):
    y = 0.5 + 0.099*x;
  if x > 5:
    y = 0.999;
  return y
"""
"""
loss = tf.where(
        condition=tf.logical_or((tf.logical_and(tf.greater(y_true, 0.0), tf.greater(y_pred, y_true))), (tf.logical_and(tf.less(y_true, 0.0), tf.less(y_pred, y_true)))),
        x=loss * penalty / 5,
        y=loss * penalty / 10
    )
"""
"""
# working
# tf.where(tf.less(x, 0), tf.add(0.5,tf.multiply(0.099,x)) ,tf.add(0.5,tf.multiply(0.099,x)))
"""
# tf.where(tf.less(x, 0), true_fn=lambda: tf.add(0.5,tf.multiply(0.099,x)), false_fn=lambda: tf.add(0.5,tf.multiply(0.099,x))) 
# tf.cond(tf.logical_and(tf.less(x,-10),tf.greater(x,-5))): 
"""
    if x > -5 and x < 5:
        return tf.add(0.5,tf.multi(0.099,x)) 
    if x > 5:
        return 0.999
"""
get_custom_objects().update({'custom_activation': Activation(custom_activation)})
"""
# Usage
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation(custom_activation, name='SpecialActivation'))
print(model.summary())
"""

"\n# Usage\nmodel = Sequential()\nmodel.add(Dense(32, input_dim=784))\nmodel.add(Activation(custom_activation, name='SpecialActivation'))\nprint(model.summary())\n"

In [26]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import tensorflow
tensorflow.random.set_seed(22)

# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

# use functional API to build cnn layers
inputs = Input(shape=input_shape)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='custom_activation')(inputs)

y = MaxPooling2D()(y)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='custom_activation')(y)

y = MaxPooling2D()(y)
# y = Conv2D(filters=filters,
#            kernel_size=kernel_size,
#            activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
#y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)



# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()
# Retrieve the config

# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=25,
          batch_size=batch_size)

# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_23 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 1600)              0  

In [27]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import tensorflow
tensorflow.random.set_seed(22)

# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

# use functional API to build cnn layers
inputs = Input(shape=input_shape)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='sigmoid')(inputs)

y = MaxPooling2D()(y)

y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='sigmoid')(y)

y = MaxPooling2D()(y)
# y = Conv2D(filters=filters,
#            kernel_size=kernel_size,
#            activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
#y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()

# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=25,
          batch_size=batch_size)

# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 1600)              0  